In [ ]:
// Nuget Packages
#r "nuget: MelonChart.NET, 2.*"

#r "nuget: Microsoft.Extensions.Configuration.Json, 8.*"
#r "nuget: Microsoft.SemanticKernel, 1.*"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.*"
#r "nuget: Microsoft.SemanticKernel.Core, 1.*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.*-*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.*-*"

#r "nuget: System.Linq.Async, 6.*"


In [ ]:
// Add using statements
using System.ComponentModel;
using System.IO;
using System.Net.Http;
using System.Text.Encodings.Web;
using System.Text.Json;
using System.Text.Json.Serialization;

using MelonChart.Models;

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;

using Kernel = Microsoft.SemanticKernel.Kernel;

In [ ]:
// Azure OpenAI configurations
var path = Path.Combine(Directory.GetCurrentDirectory(), "appsettings.Development.json");
var config = new ConfigurationBuilder().AddJsonFile(path).Build();

var endpoint = config["Azure:OpenAI:Endpoint"];
var apiKey = config["Azure:OpenAI:ApiKey"];
var deploymentName = config["Azure:OpenAI:DeploymentName"];

In [ ]:
// Build Semantic Kernel
var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion(
                       deploymentName: deploymentName,
                       endpoint: endpoint,
                       apiKey: apiKey)
                   .Build();

In [ ]:
// Import prompts
var prompts = kernel.ImportPluginFromPromptDirectory("Prompts");

In [ ]:
// Import codes
#!import Plugins/MelonChartMemory/MelonChartPlugin.cs

In [ ]:
// Import plugins
kernel.ImportPluginFromType<MelonChartPlugin>();

In [ ]:
// Build Semantic Memory
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0010
#pragma warning disable SKEXP0050

var memory = new MemoryBuilder()
                 .WithAzureOpenAITextEmbeddingGeneration(
                     deploymentName: "model-textembeddingada002-2",
                     endpoint: endpoint,
                     apiKey: apiKey)
                 .WithMemoryStore(new VolatileMemoryStore())
                 .Build();

In [ ]:
// Add HttpClient instance.
var http = new HttpClient()
{
    Timeout = TimeSpan.FromSeconds(240)
};

In [ ]:
// Add JsonSerializerOptions instance.
var jso = new JsonSerializerOptions()
{
    WriteIndented = false,
    PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
    Encoder = JavaScriptEncoder.UnsafeRelaxedJsonEscaping,
    Converters = { new JsonStringEnumConverter(JsonNamingPolicy.CamelCase) },
};

In [ ]:
// Invoke the plugin - Add Melon Chart data
await kernel.InvokeAsync(
    pluginName: nameof(MelonChartPlugin),
    functionName: nameof(MelonChartPlugin.AddMelonChart), 
    arguments: new KernelArguments()
    {
        { "memory", memory }, 
        { "http", http },
        { "jso", jso }, 
    }
);

In [ ]:
// Invoke the plugin - Add Spotify Chart data
await kernel.InvokeAsync(
    pluginName: nameof(MelonChartPlugin),
    functionName: nameof(MelonChartPlugin.AddSpotifyChart), 
    arguments: new KernelArguments()
    {
        { "memory", memory }, 
        { "http", http },
        { "jso", jso }, 
    }
);

In [ ]:
// User input
var question = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("지금 무슨 생각을 하고 계세요?");

Console.WriteLine($"User: {question}");

In [ ]:
// Invoke the prompt - Get Sentiments
var sentiments = await kernel.InvokeAsync<string>(
                       function: prompts["GetSentiments"],
                       arguments: new KernelArguments()
                       {
                           { "input", question }
                       });

Console.WriteLine(sentiments);

In [ ]:
// Invoke the prompt - Measure Sentiments
var measured = await kernel.InvokeAsync<string>(
                     function: prompts["MeasureSentiments"],
                     arguments: new KernelArguments()
                     {
                         { "input", question },
                         { "sentiments", sentiments },
                     });

Console.WriteLine(measured);

In [ ]:
// Invoke the prompt - Pick Primary Sentiment
var sentiment = await kernel.InvokeAsync<string>(
                     function: prompts["PickPrimarySentiment"],
                     arguments: new KernelArguments()
                     {
                         { "input", question },
                         { "measured", measured },
                     });

Console.WriteLine(sentiment);

In [ ]:
// Invoke the plugin - Find songs
var results = await kernel.InvokeAsync(
    pluginName: nameof(MelonChartPlugin),
    functionName: nameof(MelonChartPlugin.FindSongs), 
    arguments: new KernelArguments()
    {
        { "memory", memory }, 
        { "question", refined },
        { "jso", jso }, 
    }
);

var data = results.GetValue<List<ChartItem>>().Select(p => JsonSerializer.Serialize(p, jso)).Aggregate((x, y) => $"{x}\n{y}");

Console.WriteLine(data);